In [ ]:
import DRLPDE_main
import time

start = time.time()
DRLPDE_main.maintraining(param='JCPexample5' , use_cuda=True)

end = time.time()
print('Time to train: {:3.1f} seconds'.format(end - start))

Pre-processing: Loading parameters from JCPexample5.py
Initializing the model
Training has begun
No errors in first epoch: Training will continue
step = 2000/20000, 0.132 s/step, time-to-go:2369s
step = 4000/20000, 0.134 s/step, time-to-go:2140s
step = 6000/20000, 0.135 s/step, time-to-go:1894s
step = 8000/20000, 0.136 s/step, time-to-go:1630s
step = 10000/20000, 0.135 s/step, time-to-go:1351s
step = 12000/20000, 0.137 s/step, time-to-go:1093s
step = 14000/20000, 0.136 s/step, time-to-go:816s
step = 16000/20000, 0.136 s/step, time-to-go:543s
step = 18000/20000, 0.135 s/step, time-to-go:271s
step = 20000/20000, 0.135 s/step, time-to-go: 0s
model saved as savedmodels/JCPexample5.pt
Time to train: 2705.2 seconds


In [ ]:
import DRLPDE_postprocessing
DRLPDE_postprocessing.postprocessing(param='JCPexample2',use_model='JCPexample1')